# ML Experiment 1 - HSE CC PQ Incidents Prediction (June '25)

## Load Libraries

In [0]:
!python --version

Python 3.12.3


In [0]:
!python -m pip install --upgrade pip
!python -m pip install autogluon imbalanced-learn loguru
!pip install -U ray[data,train,tune,serve]==2.39.0
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install -U fastai==2.7.19

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
DEPRECATION: Using the pkg_resources metadata backend is deprecated. pip 26.3 will enforce this behaviour change. A possible replacement is to use the default importlib.metadata backend, by unsetting the _PIP_USE_IMPORTLIB_METADATA environment variable. Discussion can be found at https://github.com/pypa/pip/issues/13317
  Preparing metadata (setup.py) ... - \ | done
  Preparing metadata (setup.py) ... done
  Using cached lightning_utilities-0.14.3-py3-none-any.whl.metadata (5.6 kB)
  Preparing metadata (setup.py) ... - done

*** WARNING: max output size exceeded, skipping output. ***

  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━ 14/17 [ray]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━ 14/17 [ray]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━ 14/17 [ray]
   ━━━━━━━━━━━━━━━━

In [0]:
dbutils.library.restartPython()

In [0]:
import mlflow; mlflow.autolog(disable=True)

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from autogluon.tabular import TabularDataset, TabularPredictor

from imblearn.over_sampling import SMOTE

from loguru import logger
from collections import Counter

## Classification with SMOTE

In [0]:
# Load ML dataset
ml_df_class = spark.table('hive_metastore.default.hse_cc_pred_ml_2').toPandas().drop(columns=['number_of_near_miss_incidents'])

# Create training and test datasets
label = 'is_near_miss'
keys = ['production_date']
X = ml_df_class.drop(columns=[label]+keys)
y = ml_df_class[label]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

# Use standard scaling
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# Solve class imbalance using SMOTE
# logger.info(f'Class distribution before SMOTE: {Counter(y_train)}')
# sm = SMOTE(random_state=42)
# X_train, y_train = sm.fit_resample(X_train, y_train)
# logger.info(f'Class distribution after SMOTE: {Counter(y_train)}')

# Prepare the datasets for Autogluon
train_data = pd.DataFrame(X_train, columns=X.columns)
train_data[label] = y_train.values

test_data = pd.DataFrame(X_test, columns=X.columns)
test_data[label] = y_test.values

train_data_auto = TabularDataset(train_data)
test_data_auto = TabularDataset(test_data)

logger.info(f"Train data shape: {train_data_auto.shape}")
logger.info(f"Test data shape: {test_data_auto.shape}")

2025-06-23 06:05:12.589 | INFO     | __main__:<module>:32 - Train data shape: (3949, 26)
2025-06-23 06:05:12.590 | INFO     | __main__:<module>:33 - Test data shape: (1693, 26)


In [0]:
# label = 'is_near_miss'
# train_data_auto = spark.table('hive_metastore.groupdb_rakona.hse_cc_inc_pred_ml_train').toPandas()
# test_data_auto = spark.table('hive_metastore.groupdb_rakona.hse_cc_inc_pred_ml_test').toPandas()

# print(f"Train data shape: {train_data_auto.shape}")
# print(f"Test data shape: {test_data_auto.shape}")

In [0]:
predictor = TabularPredictor(label=label, eval_metric="f1").fit(
    train_data_auto,
    # time_limit=600,
    presets="best_quality"
    # num_bag_folds=5,
    # num_stack_levels=2
)

No path specified. Models will be saved in: "AutogluonModels/ag-20250623_060512-001"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.12.3
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #33~22.04.1-Ubuntu SMP Fri Apr 25 06:39:10 UTC 2025
CPU Count:          8
Memory Avail:       39.85 GB / 57.39 GB (69.4%)
Disk Space Avail:   10.00 GB / 10.00 GB (100.0%)
	We recommend a minimum available disk space of 10 GB, and large datasets may require more.
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequen

In [0]:
predictor.leaderboard(test_data_auto, extra_metrics=['accuracy', 'precision', 'recall'])

/databricks/python/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/databricks/python/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/databricks/python/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/databricks/python/lib/python3.12/site-packages/s

,model,score_test,accuracy,precision,recall,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetFastAI_r172_BAG_L1,0.250000,0.978736,0.333333,0.200000,0.209677,f1,2.225834,0.139336,10.170036,2.225834,0.139336,10.170036,1,True,74
1,NeuralNetFastAI_r145_BAG_L2,0.227273,0.979917,0.357143,0.166667,0.266667,f1,19.754344,1.760209,61.883509,2.712040,0.452743,11.225957,2,True,113
2,NeuralNetFastAI_r11_BAG_L1,0.218182,0.974601,0.240000,0.200000,0.196721,f1,2.910932,0.249604,25.303378,2.910932,0.249604,25.303378,1,True,29
3,NeuralNetFastAI_r187_BAG_L2,0.217391,0.978736,0.312500,0.166667,0.279070,f1,19.010379,1.478873,60.331643,1.968075,0.171407,9.674091,2,True,174
4,NeuralNetFastAI_r111_BAG_L2,0.212766,0.978145,0.294118,0.166667,0.230769,f1,19.353971,1.403149,55.148122,2.311668,0.095683,4.490570,2,True,142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,NeuralNetFastAI_r172_BAG_L2,0.000000,0.979917,0.000000,0.000000,0.227545,f1,19.490495,1.384457,55.870710,2.448192,0.076990,5.213158,2,True,162
174,XGBoost_r49_BAG_L2,0.000000,0.982280,0.000000,0.000000,0.135135,f1,19.507250,1.368252,56.086085,2.464946,0.060785,5.428533,2,True,147
175,XGBoost_r31_BAG_L2,0.000000,0.982280,0.000000,0.000000,0.000000,f1,19.688185,1.348887,52.401503,2.645881,0.041421,1.743951,2,True,152
176,XGBoost_r98_BAG_L2,0.000000,0.982280,0.000000,0.000000,0.000000,f1,19.822461,1.409595,54.803627,2.780158,0.102129,4.146075,2,True,130


In [0]:
predictor.evaluate(test_data_auto, model='NeuralNetFastAI_r172_BAG_L1', silent=True)

{'f1': np.float64(0.25),
 'accuracy': 0.9787359716479622,
 'balanced_accuracy': np.float64(0.5963920625375827),
 'mcc': np.float64(0.2479901107382674),
 'roc_auc': np.float64(0.8358689116055321),
 'precision': np.float64(0.3333333333333333),
 'recall': np.float64(0.2)}

In [0]:
predictor.feature_importance(test_data_auto, model='NeuralNetFastAI_r172_BAG_L1')

These features in provided data are not utilized by the predictor and will be ignored: ['breakdowns']
Computing feature importance via permutation shuffling for 24 features using 1693 rows with 5 shuffle sets...
	159.9s	= Expected runtime (31.98s per shuffle set)
	10.72s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
line_name,0.179412,0.064438,1.694764e-03,5,0.312090,0.046733
duration_sum,0.148413,0.024896,9.155540e-05,5,0.199674,0.097152
total_defects,0.117826,0.001588,3.954362e-09,5,0.121095,0.114557
unplanned_stops_per_day,0.113961,0.017270,6.138978e-05,5,0.149519,0.078403
unplanned_count,0.102584,0.019537,1.505244e-04,5,0.142812,0.062357
department_name,0.060713,0.031597,6.339937e-03,5,0.125771,-0.004345
total_defects_yesterday,0.059966,0.036252,1.042859e-02,5,0.134609,-0.014677
num_resolved_in_24_hours,0.054402,0.021600,2.445619e-03,5,0.098877,0.009926
completed_perc,0.052102,0.030706,9.599478e-03,5,0.115325,-0.011121
safe_percentage,0.047664,0.033529,1.678762e-02,5,0.116701,-0.021372
